# Data Exploration
Since the data dictionary given isnt much to go on, just some data exploration, deductions and inferences have to be made first.

### Problem Statement:  
The aim is to provide a report of the following indicators: Gross Written Premium cancelled and Gross Written Premium booked. Both indicators will have to be broken down by month and line of business (LOB).

Note that this report should be run automatically every end of the calendar month. You should not have to specify in the code which month to update: at every end of month, the report needs to show the current year figures (up to the current month) and last year for the same time period.


Definition:

Gross Written Premiums (GWP) GWP is an industry measure of the life insurance premiums due and the general insurance premiums underwritten in the reporting period, before any deductions for reinsurance.

-          YrM: transaction year and month under format YYYYMM
-          CHDRNUM: policy number
-          TRANNO: transaction number

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
##importing the xlsx and reading the sheets into dataframes
xls = pd.ExcelFile('PREMIUM_POLICY_DATA.xlsx')
policy = pd.read_excel(xls, 'POLICY')
premium = pd.read_excel(xls, 'PREMIUM')

In [3]:
#Quick look at the policy table
policy.head()

,CHDRNUM,lob,CNTTYPE,FUNDCODE,ACSTYP,D_latestexpiry
0,10201150,017-Travel,ATP,L,NO,2021-09-30
1,10201150,017-Travel,ATP,L,NO,2021-09-30
2,10201150,017-Travel,ATP,L,NO,2021-09-30
3,10201150,017-Travel,ATP,L,NO,2021-09-30
4,10201150,017-Travel,ATP,L,NO,2021-09-30


In [4]:
#Quick look at the premium table
premium.head()

,CHDRNUM,TRANNO,AGENTID,YrM,D_tran,D_eff,D_com,D_exp,TRANTYPE,FINANCIAL,GWP
0,10201150,1897,86,201912,2019-12-20,2019-11-03,2019-10-01,2020-09-30,ENDO,YES,0.0
1,10201150,1898,86,201912,2019-12-26,2019-12-15,2019-10-01,2020-09-30,ENDO,YES,149.9
2,10201150,1899,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,87.1
3,10201150,1900,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,-89.7
4,10201150,1901,86,201912,2019-12-28,2019-09-29,2018-10-01,2019-09-30,ENDO,YES,30.0


In [5]:
#Checking the Shape of the table
policy.shape

(28398, 6)

In [6]:
#Length of table happens to be the same
premium.shape

(28398, 11)

After a quick look through the xlsx, it can be ntoed that the indexes are exactly the same and as such the tables can be combined for easier exploration.

In [7]:
#concatenation works best as the column of policy numbers are alphanumeric
data = pd.concat([policy, premium], axis=1, sort=False)

In [8]:
#checking to see if it was successful
data.shape

(28398, 17)

In [9]:
#Column names check
data.columns.tolist()

['CHDRNUM',
 'lob',
 'CNTTYPE',
 'FUNDCODE',
 'ACSTYP',
 'D_latestexpiry',
 'CHDRNUM',
 'TRANNO',
 'AGENTID',
 'YrM',
 'D_tran',
 'D_eff',
 'D_com',
 'D_exp',
 'TRANTYPE',
 'FINANCIAL',
 'GWP']

In [10]:
#extracting what deems to be necessary
impt = data[['CHDRNUM',
 'lob',
 'D_latestexpiry',
 'TRANNO',
 'YrM',
 'D_tran',
 'D_eff',
 'D_com',
 'D_exp',
'TRANTYPE',
 'GWP']]

In [11]:
#column names
impt.columns

Index(['CHDRNUM', 'CHDRNUM', 'lob', 'D_latestexpiry', 'TRANNO', 'YrM',
       'D_tran', 'D_eff', 'D_com', 'D_exp', 'TRANTYPE', 'GWP'],
      dtype='object')

In [12]:
#Policy Number appeared twice therefore one of them can be renamed and dropped.
#It shouldnt be used as an index as the policy number is not unique per entry
impt.columns = ['CHDRNUM', '2', 'lob', 'D_latestexpiry', 'TRANNO', 'YrM',
       'D_tran', 'D_eff', 'D_com', 'D_exp','TRANTYPE', 'GWP']

In [13]:
#dropping the extra policy column
impt=impt[['CHDRNUM',
 'lob',
 'D_latestexpiry',
 'TRANNO',
 'YrM',
 'D_tran',
 'D_eff',
 'D_com',
 'D_exp',
'TRANTYPE',
 'GWP']]

In [14]:
#checking if the shape is still alright, with no loss in rows
impt.shape

(28398, 11)

In [15]:
#identfying the different types of transactions
impt['TRANTYPE'].unique()

array(['ENDO', 'RNWL', 'CANC', 'NWBS', 'REIN'], dtype=object)

In [16]:
#since there are 0 and negative values, filtering all negative values in the dataframe
filtered = impt[impt['GWP']<0]
#transaction types that are negative
filtered['TRANTYPE'].unique()

array(['ENDO', 'CANC'], dtype=object)

In [17]:
#transaction types that have 0 GWP
zero = impt[impt['GWP']==0]
zero['TRANTYPE'].unique()

array(['ENDO', 'RNWL', 'CANC', 'REIN', 'NWBS'], dtype=object)

Only Endo and Canc have GWP values that are negative

In [18]:
#ENDO type do have positive GWP right off the bat, filtering by CANC
filtered_2 = impt[impt['TRANTYPE']=='CANC']
#there are no positive GWP values for TRANTYPE = CANC
filtered_2[filtered_2['GWP']>0]

,CHDRNUM,lob,D_latestexpiry,TRANNO,YrM,D_tran,D_eff,D_com,D_exp,TRANTYPE,GWP


In [19]:
#There are 372 cancelled Transactions
cancelled = impt[impt['TRANTYPE']=='CANC']
cancelled.shape

(372, 11)

In [20]:
impt['YrM'].sort_values(ascending=True)

0        201912
24946    201912
24945    201912
24944    201912
24943    201912
          ...  
3951     202002
10579    202002
10572    202002
23064    202002
28397    202002
Name: YrM, Length: 28398, dtype: int64

In [21]:
impt['D_tran'].sort_values(ascending=True)

0       2019-12-20
23039   2019-12-20
24863   2019-12-20
24864   2019-12-20
24865   2019-12-20
           ...    
24192   2020-02-26
24193   2020-02-26
24194   2020-02-26
24180   2020-02-26
28397   2020-02-26
Name: D_tran, Length: 28398, dtype: datetime64[ns]

In [22]:
impt['D_eff'].sort_values(ascending=True)

1446    2016-05-01
1447    2016-08-04
1448    2017-08-04
23991   2017-11-01
23069   2017-11-01
           ...    
24715   2020-08-08
24604   2020-09-16
24667   2020-10-23
24668   2020-10-23
27334   2020-11-10
Name: D_eff, Length: 28398, dtype: datetime64[ns]

In [23]:
impt['D_com'].sort_values(ascending=True)

1446    2016-05-01
1447    2016-08-04
1448    2017-08-04
814     2017-10-01
23991   2017-11-01
           ...    
24713   2020-08-08
24604   2020-09-16
24667   2020-10-23
24668   2020-10-23
27334   2020-11-10
Name: D_com, Length: 28398, dtype: datetime64[ns]

In [24]:
impt['D_exp'].sort_values(ascending=True)

1446    2016-08-03
1447    2017-08-03
2200    2018-03-17
1448    2018-08-03
23991   2018-10-31
           ...    
24033   2022-01-22
24031   2022-01-30
23980   2022-02-22
24029   2022-02-26
23981   2022-03-01
Name: D_exp, Length: 28398, dtype: datetime64[ns]

In [25]:
travel = impt[impt['lob']== '017-Travel']

In [26]:
accident = impt[impt['lob']== '018-Personal Accident']

In [27]:
travel.shape

(6222, 11)

In [28]:
accident.shape

(22176, 11)

Summary of exploration:  
- 3 months worth of transactions  
- 2 types of LOB: Travel and PA
- ENDO and CANC are the only negative GWP
- Information utilised will be:
    - Policy table:
        - Policy Number
        - LOB
    - Premium Table:
        - Transaction Type
        - GWP
        - YrM

Deductions and Assumptions:  
- GWP Cancelled will be a SUM of the GWP where TRANTYPE = CANC sorted by LOB and by Month
- GWP Booked will be a SUM of the GWP where TRANTYPE != CANC sorted by LOB and by Month
- Hence utilising all of the data available
- All are correct and no Null
- Year and Month will be extracted from the transaction date from the YrM
- D_tran, D_eff, D_com, D_exp are unnecessary for this analysis

Table will be in the below format:

year|month|LOB|GWP Cancelled|GWP Booked
---|---|---|---|---